# 한국인만 알아들을 수 있는 글 번역하기!

In [ ]:
from IPython.display import Image
Image(filename='data/korean.jpg') 

## 1. Image to Text
-출처: https://apidocs.ncloud.com/ko/ai-application-service/ocr/ocr/

In [ ]:

import requests
import uuid
import time
import base64
import json

api_url = '<YOUR_API_URL>'
secret_key = '<YOUR_SECERET_KEY>'
# image_url = 'YOUR_IMAGE_URL'
image_file = 'data/korean.jpg'
with open(image_file,'rb') as f:
    file_data = f.read()

request_json = {
    'images': [
        {
            'format': 'jpg',
            'name': 'demo',
            'data': base64.b64encode(file_data).decode(),
        }
    ],
    'requestId': str(uuid.uuid4()),
    'version': 'V2',
    'timestamp': int(round(time.time() * 1000))
}

payload = json.dumps(request_json).encode('UTF-8')
headers = {
  'X-OCR-SECRET': secret_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", api_url, headers=headers, data = payload)



In [ ]:
result = json.loads(response.text)
extracted_text = ''
for box in result['images'][0]['fields']:
    extracted_text+=box['inferText'] +" "
extracted_text

## 2. Text to Speech
-출처: https://apidocs.ncloud.com/ko/ai-naver/clova_speech_synthesis/tts/

In [ ]:
import os
import sys
import urllib.request
client_id = "<YOUR_CLIENT_ID>"
client_secret = "<YOUR_CLIENT_SECRET>"
encText = urllib.parse.quote(extracted_text)
data = "speaker=mijin&speed=0&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/voice/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    print("TTS mp3 저장")
    response_body = response.read()
    with open('1111.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

In [ ]:
import IPython.display as ipd
ipd.Audio('1111.mp3')

## 3. Speech to Text
-출처: https://apidocs.ncloud.com/ko/ai-naver/clova_speech_recognition/stt/

In [ ]:
import sys
import requests

lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
data = open('1111.mp3', 'rb')
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}
response = requests.post(url,  data=data, headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)
